**Data preprocessing, Create Label and Features**

In [ ]:
#import dependencies
import cv2
import numpy as np
import os
from matplotlib import pyplot as pyplot
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
DATA_PATH = os.path.join("MP_DATA") # paht of the exported data
with open("word_list.txt") as file:
  action = file.readlines()
for i,word in enumerate(action):
  action[i] = word.replace("\n","")
actions = np.array(action)
label_map = {lable:num for num,lable in enumerate(actions)}
no_sequences = 30 # take 30 video to collect data for each action
sequence_length = 30

In [ ]:
sequences,lables = [], []
for action in actions:
  action_file = os.listdir(os.path.join[DATA_PATH,action])
  for sequence in range(len(action_file)):#should change to the number of file
    window = []
    for frame_num in range(sequence_length):
      res = np.load(os.path.join(DATA_PATH,action,str(sequence),f"{frame_num}.npy"))
      window.append(res)
    sequences.append(window)
    lables.append(label_map[action])
    
X = np.array(sequences)
y = to_categorical(lables).astype(int)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.05)
  

**Train the AI**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard
import tensorflowjs as tfjs

In [ ]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential() #
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,1662))) #frame and all data shape
model.add(LSTM(128,return_sequences=True,activation='relu')) 
model.add(LSTM(64,return_sequences=False,activation='relu')) # return false because next layer is dense
model.add(Dense(64,activation='relu')) 
model.add(Dense(32,activation='relu')) 
model.add(Dense(actions.shape[0],activation='softmax')) # return output by how many action do we have => [0.7,0.1,0.2] / likely be 'hello'
#loss = categorical => multiple class 
model.compile(optimizer='Adam',loss='categorical_crosentrophy',metrics=['categorical_accuracy'])
#epochs increase relatively the number of data 
model.fit(X_train,y_train,epochs=2000,callbacks=[tb_callback])

In [ ]:
#watch the log
%cd logs/train
%tensorboard --logdir=.

In [ ]:
model.summary()

**save module**

In [ ]:
model.save("action.h5")
tfjs.converters.save_keras_model(model, tfjs)

In [ ]:
# reload model weight
model.load_weights('action.h5')